In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories, plot_MC

%load_ext autoreload
%autoreload 2

Compute LiDAR error statistics

In [ ]:
N = 1900
lidar_mean_errs = []
lidar_max_errs = []

for kitti_seq in ['0018', '0027', '0028', '0034']:

    start_idx = 1550 if kitti_seq == '0028' else 0
    gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
    gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
    data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'icp')
    lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

    # Compute RMSE and max error for lidar for each sequence
    lidar_err = np.linalg.norm(lidar_positions[:N] - gt_enu[:N], axis=1)
    print("Mean error: ", np.mean(lidar_err))
    print("Max error: ", np.max(lidar_err))
    lidar_mean_errs.append(np.mean(lidar_err))
    lidar_max_errs.append(np.max(lidar_err))

Set of all runs to be considered

In [ ]:
# SR-FGO runs for each sequence and attack size
runs = [['fgo_0m_10runs_2023-01-18-1754', 'fgo_50m_5runs_2023-01-19-0042', 'fgo_100m_5runs_2023-01-19-0121', 'fgo_200m_20runs_2023-01-19-2015'], # 0018
        ['fgo_0m_10runs_2023-01-18-1843', 'fgo_50m_5runs_2023-01-19-0237', 'fgo_100m_5runs_2023-01-19-0313', 'fgo_200m_20runs_2023-01-19-2308'], # 0027
        ['fgo_0m_10runs_2023-01-18-1920', 'fgo_50m_5runs_2023-01-19-0428', 'fgo_100m_5runs_2023-01-19-0507', 'fgo_200m_20runs_2023-01-20-0138'], # 0028
        ['fgo_0m_10runs_2023-01-18-2000', 'fgo_50m_5runs_2023-01-19-0627', 'fgo_100m_5runs_2023-01-19-0707', 'fgo_200m_20runs_100w_2023-01-20-0846']] # 0034
mean_errors = np.zeros((4, 4))
max_errors = np.zeros((4, 4))
mean_errors_std = np.zeros((4, 4))
max_errors_std = np.zeros((4, 4))

# Naive-FGO runs for each sequence and attack size
runs_blind = [['fgo_0m_100runs_100w_blind', 'fgo_50m_5runs_100w_blind', 'fgo_100m_20runs_100w_blind', 'fgo_200m_20runs_100w_blind'], # 0018
        ['fgo_0m_7runs_100w_blind', 'fgo_50m_5runs_100w_blind', 'fgo_100m_20runs_100w_blind', 'fgo_200m_20runs_100w_blind'], # 0027
        ['fgo_0m_7runs_100w_blind', 'fgo_50m_5runs_100w_blind', 'fgo_100m_20runs_100w_blind', 'fgo_200m_20runs_100w_blind'], # 0028
        ['fgo_0m_6runs_100w_blind', 'fgo_50m_5runs_100w_blind', 'fgo_100m_20runs_100w_blind', 'fgo_200m_20runs_100w_blind']] # 0034
mean_errors_blind = np.zeros((4, 4))
max_errors_blind = np.zeros((4, 4))
mean_errors_blind_std = np.zeros((4, 4))
max_errors_blind_std = np.zeros((4, 4))

In [ ]:
# Compute matrix of statistics for mean and max error

traces = ['0018', '0027', '0028', '0034']
for i in range(4):
    kitti_seq = traces[i]
    start_idx = 1550 if kitti_seq == '0028' else 0
    gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
    gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
    for j in range(4):
        # SR FGO
        run = runs[i][j]
        fgo_results_path = os.path.join(os.getcwd(), '..', '..', 'results', kitti_seq, 'old', run)
        fgo_results_files = os.listdir(fgo_results_path)

        mean_errs = []
        max_errs = []
        for k in range(5):
            fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[k]))
            fgo_positions = fgo_results['positions']
            err = np.linalg.norm(fgo_positions - gt_enu[:N], axis=1)
            # print("Mean error: ", np.mean(err))
            # print("Max error: ", np.max(err))
            mean_errs.append(np.mean(err))
            max_errs.append(np.max(err))
        mean_errs_mean = np.mean(mean_errs)
        mean_errs_std = np.std(mean_errs)
        max_errs_mean = np.mean(max_errs)
        max_errs_std = np.std(max_errs)
        mean_errors[i,j] = mean_errs_mean #/ lidar_mean_errs[i]
        max_errors[i,j] = max_errs_mean #/ lidar_max_errs[i]
        mean_errors_std[i,j] = mean_errs_std
        max_errors_std[i,j] = max_errs_std

        # Naive FGO
        run_blind = runs_blind[i][j]
        fgo_results_path = os.path.join(os.getcwd(), '..', '..', 'results', kitti_seq, 'final', run_blind)
        fgo_results_files = os.listdir(fgo_results_path)

        mean_errs = []
        max_errs = []
        for k in range(5):
            fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[k]))
            fgo_positions = fgo_results['positions']
            err = np.linalg.norm(fgo_positions - gt_enu[:N], axis=1)
            # print("Mean error: ", np.mean(err))
            # print("Max error: ", np.max(err))
            mean_errs.append(np.mean(err))
            max_errs.append(np.max(err))
        mean_errs_mean = np.mean(mean_errs)
        mean_errs_std = np.std(mean_errs)
        max_errs_mean = np.mean(max_errs)
        max_errs_std = np.std(max_errs)
        mean_errors_blind[i,j] = mean_errs_mean #/ lidar_mean_errs[i]
        max_errors_blind[i,j] = max_errs_mean #/ lidar_max_errs[i]
        mean_errors_blind_std[i,j] = mean_errs_std
        max_errors_blind_std[i,j] = max_errs_std

In [ ]:
attack_rates = ['0 m/s', '0.5 m.s', '1.0 m/s', '2.0 m/s']

In [ ]:
# mean error plot
seq_i = 2
fig = go.Figure()
fig.add_trace(go.Scatter(x=attack_rates, y=4*[lidar_mean_errs[seq_i]], name='Lidar', mode='lines', line=dict(color='green', width=2, dash='dash')))
fig.add_trace(go.Scatter(x=attack_rates, y=mean_errors[seq_i], error_y=dict(type='data', array=mean_errors_std[seq_i], visible=True), 
    name='SR FGO', mode='markers', marker=dict(color='blue', size=10, symbol='diamond')))
fig.add_trace(go.Scatter(x=attack_rates, y=mean_errors_blind[seq_i], error_y=dict(type='data', array=mean_errors_blind_std[seq_i], visible=True),
    name='Naive FGO', mode='markers', marker=dict(color='red', size=10, symbol='square')))
fig.update_layout(width=500, height=500, xaxis_title='Spoofing rate', yaxis_title='Mean error (m)')
fig.update_layout(legend=dict(x=1.02, y=0.98), font=dict(size=18))

In [ ]:
# max error plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=attack_rates, y=4*[lidar_max_errs[seq_i]], name='Lidar', mode='lines', line=dict(color='green', width=2, dash='dash')))
fig.add_trace(go.Scatter(x=attack_rates, y=max_errors[seq_i], error_y=dict(type='data', array=max_errors_std[seq_i], visible=True), 
    name='SR FGO', mode='markers', marker=dict(color='blue', size=10, symbol='diamond')))
fig.add_trace(go.Scatter(x=attack_rates, y=max_errors_blind[seq_i], error_y=dict(type='data', array=max_errors_blind_std[seq_i], visible=True),
    name='Naive FGO', mode='markers', marker=dict(color='red', size=10, symbol='square')))
fig.update_layout(width=900, height=500, xaxis_title='Spoofing rate', yaxis_title='Max error (m)')
fig.update_layout(legend=dict(x=1.02, y=0.98), font=dict(size=18))

Subplots with shared axis

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=4,
                    shared_yaxes=True,
                    horizontal_spacing=0.03)

for seq_i in range(4):
    showlegend = True if seq_i == 0 else False
    fig.add_trace(go.Scatter(x=attack_rates, y=4*[lidar_max_errs[seq_i]], name='Odometry', mode='lines', line=dict(color='green', width=2, dash='dash'), 
        showlegend=showlegend), 
        row=1, col=seq_i+1)
    fig.add_trace(go.Scatter(x=attack_rates, y=max_errors[seq_i], error_y=dict(type='data', array=max_errors_std[seq_i], visible=True), 
        name='SR FGO', mode='markers', marker=dict(color='blue', size=10, symbol='diamond'), showlegend=showlegend),
        row=1, col=seq_i+1)
    fig.add_trace(go.Scatter(x=attack_rates, y=max_errors_blind[seq_i], error_y=dict(type='data', array=max_errors_blind_std[seq_i], visible=True),
        name='Naive FGO', mode='markers', marker=dict(color='red', size=10, symbol='square'), showlegend=showlegend),
        row=1, col=seq_i+1)
    
fig.update_layout(width=2000, height=500, xaxis_title='', yaxis_title='Max error (m)')
fig.update_layout(legend=dict(x=1.02, y=0.98), font=dict(size=26))

In [ ]:
fig = make_subplots(rows=1, cols=4,
                    shared_yaxes=True,
                    horizontal_spacing=0.03)

for seq_i in range(4):
    showlegend = True if seq_i == 0 else False
    fig.add_trace(go.Scatter(x=attack_rates, y=4*[lidar_mean_errs[seq_i]], name='Odometry', mode='lines', line=dict(color='green', width=2, dash='dash'), 
        showlegend=showlegend), 
        row=1, col=seq_i+1)
    fig.add_trace(go.Scatter(x=attack_rates, y=mean_errors[seq_i], error_y=dict(type='data', array=mean_errors_std[seq_i], visible=True), 
        name='SR FGO', mode='markers', marker=dict(color='blue', size=10, symbol='diamond'), showlegend=showlegend),
        row=1, col=seq_i+1)
    fig.add_trace(go.Scatter(x=attack_rates, y=mean_errors_blind[seq_i], error_y=dict(type='data', array=mean_errors_blind_std[seq_i], visible=True),
        name='Naive FGO', mode='markers', marker=dict(color='red', size=10, symbol='square'), showlegend=showlegend),
        row=1, col=seq_i+1)
    
fig.update_layout(width=2000, height=500, xaxis_title='', yaxis_title='Mean error (m)')
fig.update_layout(legend=dict(x=1.02, y=0.98), font=dict(size=26))